[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/structural-break/quickstarters/baseline/baseline.ipynb)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/master/competitions/structural-break/assets/banner.webp)

# ADIA Lab Structural Break Challenge

## Challenge Overview

Welcome to the ADIA Lab Structural Break Challenge! In this challenge, you will analyze univariate time series data to determine whether a structural break has occurred at a specified boundary point.

### What is a Structural Break?

A structural break occurs when the process governing the data generation changes at a certain point in time. These changes can be subtle or dramatic, and detecting them accurately is crucial across various domains such as climatology, industrial monitoring, finance, and healthcare.

![Structural Break Example](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/competitions/structural-break/quickstarters/baseline/images/example.png)

### Your Task

For each time series in the test set, you need to predict a score between `0` and `1`:
- Values closer to `0` indicate no structural break at the specified boundary point;
- Values closer to `1` indicate a structural break did occur.

### Evaluation Metric

The evaluation metric is [ROC AUC (Area Under the Receiver Operating Characteristic Curve)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), which measures the performance of detection algorithms regardless of their specific calibration.

- ROC AUC around `0.5`: No better than random chance;
- ROC AUC approaching `1.0`: Perfect detection.

# Setup

The first steps to get started are:
1. Get the setup command
2. Execute it in the cell below

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Reveal token](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/reveal-token.gif)

In [26]:
%pip install crunch-cli --upgrade --quiet --progress-bar off
!crunch setup-notebook structural-break FG1m0Q1GvTtTNiQa1o6ZVbmU

crunch-cli, version 6.6.1
main.py: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/22125/main.py (12140 bytes)
notebook.ipynb: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/22125/notebook.ipynb (73312 bytes)
requirements.txt: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/submissions/22125/requirements.original.txt (187 bytes)
data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_test.reduced.parquet: download from https:crunchdao--compet

# Your model

## Setup

In [27]:
import os
import typing

# Import your dependencies
import joblib
import pandas as pd
import scipy
import sklearn.metrics

In [28]:
import crunch

# Load the Crunch Toolings
crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>

cli version: 6.6.1
available ram: 12.67 gb
available cpu: 2 core
----


## Understanding the Data

The dataset consists of univariate time series, each containing ~2,000-5,000 values with a designated boundary point. For each time series, you need to determine whether a structural break occurred at this boundary point.

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

In [29]:
# Load the data simply
X_train, y_train, X_test = crunch.load_data()

data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match


### Understanding `X_train`

The training data is structured as a pandas DataFrame with a MultiIndex:

**Index Levels:**
- `id`: Identifies the unique time series
- `time`: The timestep within each time series

**Columns:**
- `value`: The actual time series value at each timestep
- `period`: A binary indicator where `0` represents the **period before** the boundary point, and `1` represents the **period after** the boundary point

In [ ]:
X_train

value  period
id    time                  
0     0     0.001858       0
      1    -0.001664       0
      2    -0.004386       0
      3     0.000699       0
      4    -0.002433       0
...              ...     ...
10000 1890 -0.005903       1
      1891  0.007295       1
      1892  0.003527       1
      1893  0.007218       1
      1894  0.000034       1

[23802099 rows x 2 columns]

### Understanding `y_train`

This is a simple `pandas.Series` that tells if a dataset id has a structural breakpoint or not.

**Index:**
- `id`: the ID of the dataset

**Value:**
- `structural_breakpoint`: Boolean indicating whether a structural break occurred (`True`) or not (`False`)

In [ ]:
y_train

id
0         True
1         True
2        False
3         True
4        False
         ...  
9996     False
9997      True
9998     False
9999     False
10000     True
Name: structural_breakpoint, Length: 10001, dtype: bool

### Understanding `X_test`

The test data is provided as a **`list` of `pandas.DataFrame`s** with the same format as [`X_train`](#understanding-X_test).

It is structured as a list to encourage processing records one by one, which will be mandatory in the `infer()` function.

In [ ]:
print("Number of datasets:", len(X_test))

Number of datasets: 101


In [ ]:
X_test[0]

value  period
id    time                  
10001 0    -0.020657       0
      1    -0.005894       0
      2    -0.003052       0
      3    -0.000590       0
      4     0.009887       0
...              ...     ...
      2517  0.005084       1
      2518 -0.024414       1
      2519 -0.014986       1
      2520  0.012999       1
      2521 -0.022138       1

[2522 rows x 2 columns]

## Strategy Implementation

There are multiple approaches you can take to detect structural breaks:

1. **Statistical Tests**: Compare distributions before and after the boundary point;
2. **Feature Engineering**: Extract features from both segments for comparison;
3. **Time Series Modeling**: Detect deviations from expected patterns;
4. **Machine Learning**: Train models to recognize break patterns from labeled examples.

The baseline implementation below uses a simple statistical approach: a t-test to compare the distributions before and after the boundary point.

### The `train()` Function

In this function, you build and train your model for making inferences on the test data. Your model must be stored in the `model_directory_path`.

The baseline implementation below doesn't require a pre-trained model, as it uses a statistical test that will be computed at inference time.

In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import t
import time
from multiprocessing import Pool, cpu_count
import statsmodels.api as sm

from scipy.stats import skew, kurtosis, iqr, ttest_ind, ks_2samp
import scipy

import joblib
import typing

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from multiprocessing import Pool, cpu_count
import warnings

import typing
import sklearn.metrics

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.distributed import DistributedSampler
from torch.nn.utils.rnn import pad_sequence
import logging
import pickle
import time
import torch.optim as optim
from torch.optim import lr_scheduler
import optuna
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from collections import OrderedDict

In [31]:

def fast_ema(x, span):
    alpha = 2/(span+1)
    ema = np.empty_like(x)
    ema[0] = x[0]
    for i in range(1, len(x)):
        ema[i] = alpha * x[i] + (1-alpha) * ema[i-1]
    return ema[-1]

def extract_features(a, b):
    features = {}

    # Basic statistics
    features['mean_a'] = np.mean(a)
    features['mean_b'] = np.mean(b)
    features['std_a'] = np.std(a)
    features['std_b'] = np.std(b)
    features['skew_a'] = skew(a)
    features['skew_b'] = skew(b)
    features['kurt_a'] = kurtosis(a)
    features['kurt_b'] = kurtosis(b)
    features['iqr_a'] = iqr(a)
    features['iqr_b'] = iqr(b)
    features['std_ratio'] = features['std_b'] / (features['std_a'] + 1e-10)  # avoid div0

    # Fit t-distribution (consider sampling if large arrays)
    params_a = scipy.stats.t.fit(a)
    params_b = scipy.stats.t.fit(b)
    features['t_val_a'] = params_a[0]
    features['t_mu_a'] = params_a[1]
    features['t_sigma_a'] = params_a[2]
    features['t_val_b'] = params_b[0]
    features['t_mu_b'] = params_b[1]
    features['t_sigma_b'] = params_b[2]

    # Differences
    features['mean_diff'] = features['mean_b'] - features['mean_a']
    features['std_diff'] = features['std_b'] - features['std_a']


    # EMA features (use fast_ema)
    ema_windows = [5, 10, 15, 20, 25]
    for w in ema_windows:
        ema_a = fast_ema(a, w)
        ema_b = fast_ema(b, w)
        features[f'ema{w}_a'] = ema_a
        features[f'ema{w}_b'] = ema_b
        features[f'ema{w}_diff'] = ema_b - ema_a

    # Cross-correlation
    min_len = min(len(a), len(b))
    a = a[:min_len]
    b = b[:min_len]
    max_lag = min(10, min_len - 1)
    xcorr = []
    for lag in range(1, max_lag + 1):
        corr = np.corrcoef(a[:-lag], b[lag:])[0, 1]
        xcorr.append(corr)

    features['max_xcorr'] = np.nanmax(xcorr)
    features['mean_xcorr'] = np.nanmean(xcorr)

    return features


In [93]:
# === Your model definitions (from your code above) ===

class LSTMEncoder(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=64, latent_dim=32, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, lengths):
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        _, (h_n, _) = self.lstm(packed)
        last_hidden = h_n[-1]  # shape: (B, hidden_dim)
        z = self.fc(last_hidden)  # shape: (B, latent_dim)
        return z

class LSTMDecoder(nn.Module):
    def __init__(self, latent_dim=32, hidden_dim=64, output_dim=2, num_layers=1):
        super().__init__()
        self.fc = nn.Linear(latent_dim, hidden_dim)
        self.lstm = nn.LSTM(output_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.output_proj = nn.Linear(hidden_dim, output_dim)

    def forward(self, z, target_len):
        batch_size = z.size(0)
        hidden = self.fc(z).unsqueeze(0).repeat(3, 1, 1)  # (num_layers, B, hidden_dim)
        cell = torch.zeros_like(hidden)

        decoder_input = torch.zeros(batch_size, target_len, 2, device=z.device)
        out, _ = self.lstm(decoder_input, (hidden, cell))
        out = self.output_proj(out)
        return out

class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=64, latent_dim=32, num_layers=3):
        super().__init__()
        self.encoder = LSTMEncoder(input_dim, hidden_dim, latent_dim, num_layers)
        self.decoder = LSTMDecoder(latent_dim, hidden_dim, input_dim, num_layers)

    def forward(self, x, lengths):
        z = self.encoder(x, lengths)
        max_len = x.size(1)
        reconstructed = self.decoder(z, max_len)
        return reconstructed, z

def regime_autoencoder(params=None, **kwargs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMAutoencoder().to(device)
    return model

In [106]:
def extract_series(X_train, device=None, logger=None):
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
    start_time = time.time()

    index_x = X_train.index.get_level_values(0).unique().tolist()
    raw_series0, raw_series1 = [], []

    for i in index_x:
        if (i % 100 == 0) and logger:
            logger.info(f"Estimating Kalman series for {i}-th series, time taken: {time.time() - start_time:.2f}s")
            start_time = time.time()

        series = X_train.loc[i]
        t0 = torch.tensor(series[series.period == 0].value.values, dtype=torch.float32).to(device)
        t1 = torch.tensor(series[series.period == 1].value.values, dtype=torch.float32).to(device)
        raw_series0.append(t0)
        raw_series1.append(t1)

    return raw_series0, raw_series1

def collate_fn(batch):
    if len(batch[0]) == 3:
        s0, s1, labels = zip(*batch)
        labels = torch.stack(labels)
    else:
        s0, s1 = zip(*batch)
        labels = None

    # Get actual lengths
    len0 = torch.tensor([len(x) for x in s0])
    len1 = torch.tensor([len(x) for x in s1])

    # Use min length to truncate all
    min_len = min(min(len0), min(len1))
    s0_trunc = [x[:min_len] for x in s0]
    s1_trunc = [x[:min_len] for x in s1]

    # Update lengths since everything is truncated to min_len
    updated_lengths = torch.full((len(s0),), fill_value=min_len, dtype=torch.long)

    # Stack and combine
    pad0 = torch.stack(s0_trunc).unsqueeze(-1)  # (B, T, 1)
    pad1 = torch.stack(s1_trunc).unsqueeze(-1)  # (B, T, 1)
    combined = torch.cat([pad0, pad1], dim=-1)  # (B, T, 2)

    if labels is not None:
        return combined, updated_lengths, labels
    else:
        return combined, updated_lengths

# === Custom Dataset ===
class RegimePairDataset(Dataset):
    def __init__(self, raw_0, raw_1, labels=None):
        self.data = []
        self.has_labels = labels is not None

        if self.has_labels:
            for r0, r1, label in zip(raw_0, raw_1, labels):
                r0 = r0.cpu() if isinstance(r0, torch.Tensor) else torch.tensor(r0, dtype=torch.float32)
                r1 = r1.cpu() if isinstance(r1, torch.Tensor) else torch.tensor(r1, dtype=torch.float32)
                self.data.append((r0, r1, torch.tensor(label, dtype=torch.float32)))
        else:
            for r0, r1 in zip(raw_0, raw_1):
                r0 = r0.cpu() if isinstance(r0, torch.Tensor) else torch.tensor(r0, dtype=torch.float32)
                r1 = r1.cpu() if isinstance(r1, torch.Tensor) else torch.tensor(r1, dtype=torch.float32)
                self.data.append((r0, r1))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


# === DataLoader Creator ===
def create_data_loaders(X_train,y_train=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    r0, r1 = extract_series(X_train, device=device, logger=None)
    pair_dataset = RegimePairDataset(r0, r1, y_train)

    train_size = int(0.8*len(pair_dataset))
    val_size = len(pair_dataset) - train_size
    generator = torch.Generator().manual_seed(42)

    train_dataset, val_dataset = random_split(pair_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=64, sampler=None,
                              num_workers=max(2, cpu_count()), pin_memory=True, persistent_workers=True, prefetch_factor=4,
                              collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=64, sampler=None,
                           num_workers=max(2, cpu_count()), pin_memory=True, persistent_workers=True, prefetch_factor=4,
                            collate_fn=collate_fn)

    complete_loader = DataLoader(pair_dataset, batch_size=64, sampler=None,
                           num_workers=max(2, cpu_count()), pin_memory=True, persistent_workers=True, prefetch_factor=4,
                            collate_fn=collate_fn)

    return train_loader, val_loader, complete_loader


def create_test_data_loaders(X_test):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    r0, r1 = extract_series(X_test, device=device, logger=None)
    pair_dataset = RegimePairDataset(r0, r1)

    complete_loader = DataLoader(pair_dataset, batch_size=64, sampler=None,
                           num_workers=0, pin_memory=True,# persistent_workers=True, prefetch_factor=4,
                            collate_fn=collate_fn)

    return complete_loader


In [107]:
cpu_count()

2

In [109]:
class Trainer:
    def __init__(self, model, optimizer, scheduler, loss_func, train_data_loader, val_data_loader, dir_name):
        os.makedirs(dir_name, exist_ok=True)
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.loss_func = loss_func
        self.train_data_loader = train_data_loader
        self.val_data_loader = val_data_loader
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.logs = {}
        self.startEpoch = 0
        self.epoch = 0
        self.iters = 0
        self.dir_name = dir_name

    def train(self):
        best_loss = float('inf')
        best_epoch = 0
        self.logs['best_epoch'] = best_epoch
        self.logs['best_val_loss'] = best_loss

        for epoch in range(self.startEpoch, 100):
            self.epoch = epoch
            start = time.time()
            print(f"Starting with epoch{epoch} and time taken is {time.time() - start}")

            tr_time = self.train_one_epoch()
            val_loss, val_time = self.compute_validation_loss()

            self.scheduler.step()

            is_best_loss = val_loss < best_loss
            if is_best_loss:
                best_loss = val_loss
                best_epoch = epoch
                path = os.path.join(self.dir_name,'latest_model_vae.ckpt')
                self.save_checkpoint(path)

            self.logs['train_loss'] = self.train_loss
            self.logs['val_loss'] = val_loss
            self.logs['best_val_loss'] = best_loss
            self.logs['best_epoch'] = best_epoch

    def train_one_epoch(self):
        self.model.train()
        tr_time = 0
        total_loss = 0.0

        for i,batch in enumerate(self.train_data_loader):

            combined, lengths, labels = batch
            combined = combined.to(self.device)
            lengths = lengths.to(self.device)

            self.optimizer.zero_grad()
            tr_start = time.time()
            recon_x, _ = self.model(combined, lengths)
            loss = self.loss_func(recon_x, combined)
            loss.backward()
            self.optimizer.step()
            tr_time += time.time() - tr_start

            total_loss += loss.item()

        self.train_loss = total_loss / len(self.train_data_loader)
        return tr_time

    def compute_validation_loss(self):
        self.model.eval()
        val_loss = 0.0
        val_time = 0.0

        with torch.no_grad():
            for batch in self.val_data_loader:
                combined, lengths, labels = batch
                combined = combined.to(self.device)
                lengths = lengths.to(self.device)

                val_start = time.time()
                recon_x, _ = self.model(combined, lengths)
                loss = self.loss_func(recon_x, combined)
                val_loss += loss.item()
                val_time += time.time() - val_start

        val_loss /= len(self.val_data_loader)
        return val_loss, val_time

    def save_checkpoint(self, path):
        torch.save({
            'epoch': self.epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_val_loss': self.logs['best_val_loss'],
        }, path)


In [110]:
def process_chunk(temp):
  t0_series = temp[temp.period == 0].value.values
  t1_series = temp[temp.period == 1].value.values
  features_v = extract_features(t0_series, t1_series)
  return features_v

def process_wrapper(i_X):
    i, X = i_X
    if i%100 == 0:
        print(f"presently running {i} data item to extract")
    return i, process_chunk(X.loc[i])

In [111]:
def train(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    model_directory_path: str,
):

    os.makedirs(model_directory_path, exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_rnn = regime_autoencoder().to(device)
    train_data_loader, val_data_loader,complete_data_loader = create_data_loaders(X_train,y_train)

    optimizer = optim.Adam(model_rnn.parameters(), lr=1e-4)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

    loss_func = torch.nn.MSELoss()

    trainer = Trainer(model_rnn, optimizer, scheduler, loss_func, train_data_loader, val_data_loader,model_directory_path)
    trainer.train()

    model_rnn_updated = trainer.model

    z = []

    for batch in complete_data_loader:
        combined, lengths, labels = batch
        combined = combined.to(device)
        lengths = lengths.to(device)

        recon_x, latent = model_rnn_updated(combined, lengths)  # assuming "_" is the latent vector
        z.append(latent.detach().cpu().numpy())     # detach, move to CPU, convert to numpy

    z = np.concatenate(z, axis=0)  # shape: (total_samples, latent_dim)

    index_x = X_train.index.get_level_values(0).unique().tolist()

    print(f"Spawning {cpu_count()} parallel workers...")

    # Prepare iterable of (i, X) tuples
    args = [(i, X_train) for i in index_x]

    with Pool(processes=cpu_count()) as pool:
        results = list(pool.imap_unordered(process_wrapper, args, chunksize=50))

    # Unpack results
    index_x, features_list = zip(*results)

    data = pd.DataFrame.from_records(features_list, index=index_x)
    data.index.name = "time"

    print(f"shape of data is {data.shape} and len is {len(index_x)}")

    z_df = pd.DataFrame(z, index=data.index)  # ensure the index matches

    df_new = pd.concat([data, z_df], axis=1)

    n_trials = 200   # number of Optuna trials
    cv = 3

    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500,step=100),
            'max_depth': trial.suggest_int('max_depth', 3, 18,step=3),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),
            'subsample': trial.suggest_float('subsample', 0.8, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.8, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 1),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1.0, 3.0),
            'scale_pos_weight': trial.suggest_categorical('scale_pos_weight', [1, 3, 5, 10]),
            'eval_metric': 'auc',
            'use_label_encoder': False,
            'random_state': 42,
        }

        clf = XGBClassifier(**params)

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('clf', clf)
        ])

        scores = cross_val_score(pipeline, data, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
        return scores.mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)

    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # Train best model on full data
    best_params = trial.params
    best_params['eval_metric'] = 'auc'
    best_params['use_label_encoder'] = False
    best_params['random_state'] = 42

    best_clf = XGBClassifier(**best_params)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', best_clf)
    ])

    pipeline.fit(data, y_train)

    os.makedirs(model_directory_path, exist_ok=True)

    joblib.dump(pipeline, os.path.join(model_directory_path, 'model_with_rnn_results.joblib'))


### The `infer()` Function

In the inference function, your trained model (if any) is loaded and used to make predictions on test data.

**Important workflow:**
1. Load your model;
2. Use the `yield` statement to signal readiness to the runner;
3. Process each dataset one by one within the for loop;
4. For each dataset, use `yield prediction` to return your prediction.

**Note:** The datasets can only be iterated once!

In [112]:
def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    model_xgb = joblib.load(os.path.join(model_directory_path, 'model_with_rnn_results.joblib'))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Load the model
    model_rnn = regime_autoencoder(model_directory_path,).to(device)

    checkpoint_path = os.path.join(model_directory_path,'latest_model_vae.ckpt')
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Attempt to load the model state dict
    try:
        model_rnn.load_state_dict(checkpoint['model_state_dict'])

    except RuntimeError as e:
        print("RuntimeError caught, trying to fix state dict keys...")
        new_state_dict = OrderedDict()
        for key, val in checkpoint['model_state_dict'].items():
            new_key = key
            if key.startswith("module."):  # typical of DDP-wrapped models
                new_key = key[7:]
            new_state_dict[new_key] = val
        model_rnn.load_state_dict(new_state_dict)

    model_rnn = model_rnn.eval()

    yield  # Mark as ready

    # X_test can only be iterated once.
    # Before getting the next dataset, you must predict the current one.
    for dataset in X_test:
      test_dataset = create_test_data_loaders(dataset)
      z = []

      for batch in test_dataset:
          combined, lengths, labels = batch
          combined = combined.to(device)
          lengths = lengths.to(device)

          recon_x, latent = model_rnn(combined, lengths)  # assuming "_" is the latent vector
          z.append(latent.detach().cpu().numpy())     # detach, move to CPU, convert to numpy

      z = np.concatenate(z, axis=0)  # shape: (total_samples, latent_dim)

      features = process_chunk(dataset)
      features_df = pd.DataFrame([features])
      z_df = pd.DataFrame(z, index=features_df.index)  # ensure the index matches
      df_new = pd.concat([features_df, z_df], axis=1)

      prediction = model_xgb.predict_proba(df_new)[0, 1]

      yield prediction  # Send the prediction for the current dataset



## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [113]:
crunch.test(
    # Uncomment to disable the train
    # force_first_train=False,

    # Uncomment to disable the determinism check
    # no_determinism_check=True,
)

07:12:22 no forbidden library found
07:12:22 
07:12:22 started
07:12:22 running local test
07:12:22 internet access isn't restricted, no check will be done
07:12:22 
07:12:23 starting unstructured loop...
07:12:23 executing - command=train


data/X_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match
Starting with epoch0 and time taken is 1.1920928955078125e-06
z shae is torch.Size([64, 32])
z shae is torch.Size([16, 32

[I 2025-06-08 07:13:04,709] A new study created in memory with name: no-name-7f9ff31c-6749-40d3-803f-920e56a032e8


shape of data is (100, 36) and len is 100


<ipython-input-111-38dc791312a8>:66: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),
<ipython-input-111-38dc791312a8>:71: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
<ipython-input-111-38dc791312a8>:72: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_lambda': trial.suggest_loguniform('reg_lambda', 1.0, 3.0),
[I 2025-06-08 07:13:07,697] Trial 0 finished with value: 0.4887511478

Best trial:
  Value: 0.6113406795224977
  Params: 
    n_estimators: 400
    max_depth: 15
    learning_rate: 0.0020440212605430546
    subsample: 0.9851641429784487
    colsample_bytree: 0.8875997184341492
    gamma: 0.025938127113181234
    min_child_weight: 4
    reg_alpha: 0.007625880238500161
    reg_lambda: 1.764925582678173
    scale_pos_weight: 10


07:15:00 executing - command=infer
Traceback (most recent call last):
  File "https://github.com/crunchdao/competitions/raw/refs/heads/master/competitions/structural-break/scoring/runner.py", line 31, in run
  File "/usr/local/lib/python3.11/dist-packages/crunch/runner/local.py", line 559, in execute
    result = utils.smart_call(
             ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/crunch/utils.py", line 275, in smart_call
    return function(**arguments)
           ^^^^^^^^^^^^^^^^^^^^^
  File "https://github.com/crunchdao/competitions/raw/refs/heads/master/competitions/structural-break/scoring/runner.py", line 113, in infer
  File "/usr/local/lib/python3.11/dist-packages/crunch/container.py", line 168, in __init__
    if next(consumer) is not None:
       ^^^^^^^^^^^^^^
  File "<ipython-input-112-78a0ccf751ee>", line 15, in infer
    model_rnn.load_state_dict(checkpoint['model_state'])
                              ~~~~~~~~~~^^^^^^^^^^^^^^^
KeyError: 'model

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/crunch/utils.py", line 439, in limit_traceback
    yield
  File "/usr/local/lib/python3.11/dist-packages/crunch/utils.py", line 273, in smart_call
    return function(**arguments)
           ^^^^^^^^^^^^^^^^^^^^^
  File "https://github.com/crunchdao/competitions/raw/refs/heads/master/competitions/structural-break/scoring/runner.py", line 31, in run
  File "/usr/local/lib/python3.11/dist-packages/crunch/runner/local.py", line 559, in execute
    result = utils.smart_call(
             ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/crunch/utils.py", line 275, in smart_call
    return function(**arguments)
           ^^^^^^^^^^^^^^^^^^^^^
  File "https://github.com/crunchdao/competitions/raw/refs/heads/master/competitions/structural-break/scoring/runner.py", line 113, in infer
  File "/usr/local/lib/python3.11/dist-packages/crunch/container.py", line 168, in __init__
    if next(consumer)

TypeError: object of type 'NoneType' has no len()

## Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [ ]:
prediction = pd.read_parquet("data/prediction.parquet")
prediction

### Local scoring

You can call the function that the system uses to estimate your score locally.

In [ ]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"]

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Download and Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/download-and-submit-notebook.gif)